In [5]:
import pandas as pd
import numpy as np
from tensorflow import keras
from sklearn.model_selection import train_test_split

table = {
    "age": [
        22, 23, 24, 25, 26, 27, 28, 29, 30, 31,
        32, 33, 34, 35, 36, 37, 38, 39, 40, 41
    ],
    "affordibility": [
        1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
        0, 1, 1, 1, 0, 0, 0, 1, 1, 0
    ],
    "predictedInsurance": [
        1, 1, 1, 1, 0, 1, 1, 0, 0, 0,
        0, 1, 0, 1, 0, 0, 1, 1, 0, 0
    ]
}



df=pd.DataFrame(table)
df

,age,affordibility,predictedInsurance
0,22,1,1
1,23,1,1
2,24,1,1
3,25,1,1
4,26,1,0
5,27,1,1
6,28,0,1
7,29,0,0
8,30,0,0
9,31,0,0


In [6]:
def sigmoid_numpy(n):
  return 1/(1+np.exp(-n))

In [7]:
def log_loss(y_true, y_predicted):
    epsilon = 1e-15
    y_predicted_new = [max(i,epsilon) for i in y_predicted]
    y_predicted_new = [min(i,1-epsilon) for i in y_predicted_new]
    y_predicted_new = np.array(y_predicted_new)
    return -np.mean(y_true*np.log(y_predicted_new)+(1-y_true)*np.log(1-y_predicted_new))

In [8]:
class NN:
  def __init__(self):
    self.w1=1
    self.w2=1
    self.bias=0


  def fit(self,X,y,epochs,loss_thresold):
    self.w1,self.w2,self.bias=self.gradient_descent(X['age'],X['affordibility'],y,epochs,loss_thresold)
    print(f"Final weights and bias: w1: {self.w1}, w2: {self.w2}, bias: {self.bias}")


  def predict(self,X_test):
    weighted_sum=self.w1*X_test['age']+self.w2*X_test['affordibility']+self.bias
    return sigmoid_numpy(weighted_sum)



  def gradient_descent(self,age,affordibility,y,epochs,loss_thresold):
    self.w1=1
    self.w2=1
    self.bias=0
    rate=0.1
    n=len(age)
    for i in range(epochs):
      weighted_sum=self.w1*age+self.w2*affordibility+self.bias
      y_predicted=sigmoid_numpy(weighted_sum)
      loss=log_loss(y,y_predicted)
      w1d=(1/n)*np.dot(np.transpose(age),(y_predicted-y))
      w2d=(1/n)*np.dot(np.transpose(affordibility),(y_predicted-y))

      bias_d=np.mean(y_predicted-y)
      self.w1=self.w1-rate*w1d
      self.w2=self.w2-rate*w2d
      self.bias=self.bias-rate*bias_d

      if i%50==0:
        print (f'Epoch:{i}, w1:{self.w1}, w2:{self.w2}, bias:{self.bias}, loss:{loss}')

      if loss<=loss_thresold:
        print (f'Epoch:{i}, w1:{self.w1}, w2:{self.w2}, bias:{self.bias}, loss:{loss}')
        break


    return self.w1,self.w2,self.bias

In [9]:
X_train, X_test, y_train, y_test=train_test_split(df[["age","affordibility"]],df.predictedInsurance,test_size=0.2,random_state=42)
X_train_scaled = X_train.copy()
X_train_scaled['age'] = X_train_scaled['age'] / 100

X_test_scaled = X_test.copy()
X_test_scaled['age'] = X_test_scaled['age'] / 100



In [10]:
X_test_scaled

,age,affordibility
0,0.22,1
17,0.39,1
15,0.37,0
1,0.23,1


In [11]:
customModel=NN()
customModel.fit(X_train_scaled,y_train,epochs=1000,loss_thresold=0.4631)

Epoch:0, w1:0.991442964330132, w2:0.991923269849246, bias:-0.024677953918281384, loss:0.7651710640150847
Epoch:50, w1:0.7469784113199511, w2:0.8719500771328396, bias:-0.6826068306099186, loss:0.6399266453798415
Epoch:100, w1:0.6540399843749051, w2:0.9736998485467715, bias:-0.866507308815219, loss:0.6283739727688287
Epoch:150, w1:0.5942598158713434, w2:1.0927515920612252, bias:-0.9496435662634394, loss:0.6233625741555991
Epoch:200, w1:0.5447062298248442, w2:1.1946285963821295, bias:-1.0037997569515622, loss:0.6201760919045185
Epoch:250, w1:0.5004429912737623, w2:1.2774887911732435, bias:-1.0439420570850528, loss:0.6180685201465133
Epoch:300, w1:0.45989662576205903, w2:1.3443268904029182, bias:-1.0745295525113936, loss:0.6166462287693271
Epoch:350, w1:0.4222456691290634, w2:1.398255886947978, bias:-1.097754797162487, loss:0.6156648593060421
Epoch:400, w1:0.38691881212606793, w2:1.4418443136517496, bias:-1.1151070397761564, loss:0.6149696660390567
Epoch:450, w1:0.3534773187116029, w2:1.47

In [12]:
customModel.predict(X_test_scaled)

0     0.622912
17    0.624762
15    0.249066
1     0.623021
dtype: float64

In [13]:
y_predicted_scaled = customModel.predict(X_test_scaled)
y_predicted = (y_predicted_scaled > 0.5).astype(int)
y_predicted

0     1
17    1
15    0
1     1
dtype: int32

In [14]:
y_predicted_scaled = customModel.predict(X_test_scaled)
y_predicted = (y_predicted_scaled > 0.5).astype(int)

from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_predicted)
print(f"Accuracy of the model: {accuracy}")

Accuracy of the model: 1.0
